#### Data Modelling

In [ ]:
import pandas as pd
import duckdb
from ydata_profiling import ProfileReport

In [ ]:
db_path = '../../database/nsw_doe_data_case_study.duckdb'
con = duckdb.connect(db_path)
con.execute("CREATE TABLE student_attendance_data AS SELECT * FROM read_csv_auto('../../data/attendance_data.csv');")
con.commit()
# Close the connection.
con.close()

In [ ]:
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=True)
results = con.execute("SELECT * FROM multi_age_composite_dataset LIMIT 10").fetchall()
print(results)
con.close()

In [ ]:
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=True)
table_list = con.execute("SHOW TABLES").fetchall()
print(table_list)
con.close()

In [ ]:
# Connect to database
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=True)
# Rename columns
con.execute('ALTER TABLE multi_age_composite_dataset RENAME COLUMN "School code" TO School_code')
con.execute('ALTER TABLE multi_age_composite_dataset RENAME COLUMN "School Name" TO School_Name')
years = ['14', '15', '16', '17', '18', '19', '21', '22']
measures = ['COMPOSITE_CLASS_COUNT', 'COMPOSITE_CLASS_STUDENTS', 'PCT_COMPOSITE_CLASSES', 'PCT_COMPOSITE_CLASS_STUDENTS']
sql_statements = []
for measure in measures:
    for year in years:
        sql = f"""
        SELECT 
            School_code,
            School_Name,
            '{measure}' AS Measure,
            '20{year}' AS Year,
            {measure}_{year} AS value
        FROM multi_age_composite_dataset
        """
        sql_statements.append(sql)
final_query = "\nUNION ALL\n".join(sql_statements)
con.execute(f"CREATE TABLE unpivoted_data AS {final_query}")

In [ ]:
# Establish a connection
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb', read_only=False)

# Rename the table
query = """
ALTER TABLE unpivoted_dataset RENAME TO multi_age_composite_unpivoted;
"""
con.execute(query)

# Close the connection
con.close()

In [ ]:
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=True)
df = con.sql('select * from multi_age_composite_unpivoted').df()
print(df.shape)
df.head()
con.close()

In [ ]:
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=True)
df = con.sql('select * from multi_age_composite_unpivoted').df()
profile = ProfileReport(df, title="Example Profiling Report")
profile


In [ ]:
con.close()

In [ ]:
con = duckdb.connect('../../database/nsw_doe_data_case_study.duckdb',read_only=True)
df = con.sql('select * from multi_age_composite_unpivoted').df()
profile = ProfileReport(df, title="Example Profiling Report")
profile